In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('D:\\titanic\\train.csv')

np.set_printoptions(threshold=np.inf) # unfold the display result of array
pd.options.display.max_columns = None # unfold the display result of DataFarme

import warnings
warnings.filterwarnings('ignore')

In [4]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
drop_col = ['Cabin','PassengerId','Name','Ticket']

data1 = data.drop(drop_col,axis = 1)
data2 = data1.dropna(axis = 0)

y = data2['Survived']

In [7]:
sex_oc = pd.get_dummies(data2['Sex'],prefix = 'Sex')
embarked_oc = pd.get_dummies(data2['Embarked'],prefix = 'Embarked')

In [8]:
data3 = data2.drop(['Sex','Embarked'],axis = 1)
data4 = pd.concat([data3,sex_oc,embarked_oc],axis = 1)

In [9]:
x = data4.drop(['Survived'],axis = 1)
y = data4['Survived']

In [10]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(x,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [36]:
svm.score(x,y)

0.9002808988764045

In [42]:
svm2 = SVC()
svm2.fit(x.drop(['Age'],axis = 1),y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [43]:
svm2.score(x.drop(['Age'],axis = 1),y)

0.824438202247191

In [37]:
test = pd.read_csv('D:\\titanic\\test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [38]:
test1 = test.drop(['Cabin','Name','Ticket'],axis = 1)

sex_oc_t = pd.get_dummies(test1['Sex'],prefix = 'Sex')
embarked_oc_t = pd.get_dummies(test1['Embarked'],prefix = 'Embarked')

test2 = test1.drop(['Sex','Embarked'],axis = 1)
test3 = pd.concat([test2,sex_oc_t,embarked_oc_t],axis = 1)

test3['Fare'].fillna(10,inplace = True)

In [39]:
test3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Sex_female     418 non-null uint8
Sex_male       418 non-null uint8
Embarked_C     418 non-null uint8
Embarked_Q     418 non-null uint8
Embarked_S     418 non-null uint8
dtypes: float64(2), int64(4), uint8(5)
memory usage: 21.8 KB


In [99]:
test3.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,0,1


In [92]:
result = {}

for i in range(len(test3)):
    each = test3.iloc[i][:]
    if np.isnan(each['Age']) == True:
        result[int(each['PassengerId'])] = svm2.predict([each.drop(['PassengerId','Age']).values])[0]
    else:
        result[int(each['PassengerId'])] = svm.predict([each.drop(['PassengerId']).values])[0]

In [100]:
print(result)

{892: 0, 893: 0, 894: 1, 895: 0, 896: 0, 897: 0, 898: 0, 899: 1, 900: 1, 901: 1, 902: 0, 903: 0, 904: 1, 905: 0, 906: 0, 907: 1, 908: 0, 909: 0, 910: 0, 911: 0, 912: 1, 913: 1, 914: 1, 915: 1, 916: 1, 917: 0, 918: 1, 919: 0, 920: 1, 921: 0, 922: 1, 923: 1, 924: 1, 925: 1, 926: 1, 927: 0, 928: 1, 929: 0, 930: 0, 931: 1, 932: 0, 933: 0, 934: 0, 935: 1, 936: 1, 937: 0, 938: 0, 939: 0, 940: 1, 941: 1, 942: 1, 943: 0, 944: 1, 945: 1, 946: 0, 947: 0, 948: 0, 949: 0, 950: 0, 951: 1, 952: 0, 953: 0, 954: 0, 955: 0, 956: 1, 957: 1, 958: 1, 959: 1, 960: 0, 961: 0, 962: 0, 963: 0, 964: 0, 965: 0, 966: 1, 967: 1, 968: 0, 969: 0, 970: 0, 971: 0, 972: 1, 973: 1, 974: 1, 975: 0, 976: 0, 977: 0, 978: 0, 979: 1, 980: 1, 981: 1, 982: 0, 983: 0, 984: 1, 985: 0, 986: 1, 987: 0, 988: 1, 989: 0, 990: 1, 991: 0, 992: 1, 993: 1, 994: 0, 995: 0, 996: 1, 997: 0, 998: 0, 999: 0, 1000: 0, 1001: 0, 1002: 0, 1003: 1, 1004: 1, 1005: 1, 1006: 1, 1007: 0, 1008: 0, 1009: 1, 1010: 1, 1011: 1, 1012: 1, 1013: 0, 1014: 1, 

In [98]:
import os
import csv

with open('D:\\result.csv','w') as fw:
    cw = csv.writer(fw)
    for each in result:
        cw.writerow([each,result[each]])